In [2]:
import pandas as pd
import numpy as np
from operator import itemgetter
import time
from datetime import datetime, timedelta



mec_name = "Результаты мониторингов.xlsx"
s_name = "Sheet"
# unique_name = "unique_test.xlsx"
final_name = "Результат сортировки.xlsx"
# pp_name = "pp_s.XLSX"
# sp_name = "sp_s.XLSX"

start = time.time()

# импорт настроечных таблиц (правила)
############################################################
tk_so_cl = pd.read_excel("правила.xlsx",
                        sheet_name = "ТК",
                        dtype=str) 



# импорт файла из сап портала и заполнение основного (mec)
############################################################

mec = pd.read_excel(mec_name,
                    skiprows = 0,
                        sheet_name = s_name,
                        dtype=str) 

### преобразование таблицы
## первая часть таблицы

# mec_1 = mec.iloc[:, :19].copy()
# mec_1.columns = mec_1.iloc[0]
# mec_1 = mec_1[1:].reset_index(drop=True)
# mec_1["ТК Ленты"] = mec_1["ТК Ленты"].str[-4:]

## второя половина таблицы
# берем все столбцы начина с 34
# mec_2 =  mec.iloc[:, 34:].copy()

# удаляем первую строку
# mec_2 = mec_2.drop(index=0).reset_index(drop=True)

# удаляем каждый 3 столбец с примечанием
# mec_2 = mec_2.drop(mec_2.columns[2::3], axis=1)

# перевод из str во float и округление
mec = mec.astype(float)
# mec_2 = mec_2.applymap(lambda x: round(x, 2))

# Оставляем первое слово в названии
# mec_2.rename(columns=lambda x: x.split(" ")[0], inplace=True)

# копируем название магазинов для акций
for i in range(1, len(mec_2.columns), 2):
    mec_2.columns.values[i] = mec_2.columns.values[i-1]

# добавление индекса к названию магазина
mec_2.columns = [f'{mec_2.columns[i]} {i+1}' for i, columns in enumerate(mec_2.columns)]
mec = pd.concat([mec_1, mec_2], axis=1)


## Перенос дубликатов
# объединение строк с одинаковыми товарами в одном кластере
mec_grouped = pd.DataFrame()
mec_grouped = mec.groupby(['Кластер', 'Внутренний код товара']).agg({'Тип мониторинга': ' '.join})
mec = mec.drop('Тип мониторинга', axis=1)
mec_grouped = pd.concat([mec_grouped] + [mec.groupby(['Кластер', 'Внутренний код товара'])[col].min() 
                                                   for col in mec.columns], axis=1)
mec_grouped = mec_grouped.reset_index(drop=True)



### заполнение итогового файла
f_report = mec_grouped[["Тип мониторинга", "Кластер", "ТК Ленты", 
                "Внутренний код товара","Наименование товара", "EPP", "ЧТМ"]]

f_report = f_report.rename(columns={"Наименование товара":"SAP name",
                                    "ТК Ленты":"ТК",
                                    "Внутренний код товара":"Код товара",
                                    "Тип мониторинга":"Мониторинг"})

f_report = f_report.merge(tk_so_cl[['ТК','СО','Ценовой Кластер','Группа Кластеров', '№ Города']],
                    on='ТК', how='left',)


# ипорт ПЦ (sp)
############################################################
# sp = pd.read_excel(sp_name,
#                     sheet_name = "Sheet1",
#                     dtype=str)

# pp = pd.read_excel(pp_name,
#                     sheet_name = "Sheet1",
#                     dtype=str)




# Поиск ПЦ, ЗЦ, БУ
############################################################

# def update_only_null_value(l_df, r_df,
#                            l_df_values, r_df_values, 
#                            l_key, r_key):
#     for i in range(len(l_key)):
#         r_df = r_df.rename(columns={r_key[i]:l_key[i]})

#     l_df = l_df.merge(r_df[r_df_values].drop_duplicates(subset=l_key),
#                       on=l_key, how='left', suffixes=('_l', '_r'))
#     try:
#         for i in range(len(l_df_values)):

#             l_df[str(l_df_values[i])+'_l'].fillna(l_df[str(l_df_values[i])+'_r'], inplace=True)
#             l_df = l_df.drop(columns=[str(l_df_values[i])+'_r'])
#             l_df = l_df.rename(columns={str(l_df_values[i])+'_l':str(l_df_values[i])})
#     except Exception:
#         pass
#     return l_df



# f_report = update_only_null_value(f_report, sp,
#                                   ['Цена по ДК', 'Цена по акции'], 
#                                   ['Код товара', 'ТК','Цена по ДК', 'Цена по акции'],
#                                   ['Код товара', 'ТК'], 
#                                   ['Код товара', 'ТК'])

# f_report = update_only_null_value(f_report, sp,
#                                   ['Цена по ДК', 'Цена по акции'], 
#                                   ['Код товара', 'Кластер','Цена по ДК', 'Цена по акции'],
#                                   ['Код товара', 'Кластер'], 
#                                   ['Код товара', 'Наименование кластера'])

# f_report = update_only_null_value(f_report, sp,
#                                   ['Цена по ДК', 'Цена по акции'], 
#                                   ['Код товара', 'Группа Кластеров','Цена по ДК', 'Цена по акции'],
#                                   ['Код товара', 'Группа Кластеров'], 
#                                   ['Код товара', 'Наименование Группы кластеров'])

# f_report = update_only_null_value(f_report, sp,
#                                   ['Цена по ДК', 'Цена по акции'], 
#                                   ['Код товара', 'СО','Цена по ДК', 'Цена по акции'],
#                                   ['Код товара', 'СО'], 
#                                   ['Код товара', 'Сбыт. Орг.'])
# display(f_report)

# # Поиск и сортиовка цен
# ############################################################




# # разделение таблиц для рпц и апц

last_index = mec_grouped.columns.get_loc('Дата мониторинга')
reg_pr =mec_grouped.iloc[:, last_index+1::2].copy()
# display(reg_pr)

promo_pr =mec_grouped.iloc[:, last_index+2::2].copy()
# display(promo_pr)

# reg_pr =mec_grouped.iloc[:, 3::2].copy()
# display(reg_pr)

# promo_pr =sku_list_grouped.iloc[:, 4::2].copy()
# display(promo_pr)

# promo_pr = sku_list_grouped.iloc[:, :3].copy()
# promo_pr = pd.concat([promo_pr, sku_list_grouped.iloc[:, 4::2]], axis=1)

# Оставляем первое слово в названии
# reg_pr.rename(columns=lambda x: x.split(" ")[0] if reg_pr.columns.get_loc(x) >= 3 else x, inplace=True)
# promo_pr.rename(columns=lambda x: x.split(" ")[0] if promo_pr.columns.get_loc(x) >= 3 else x, inplace=True)

# reg_pr.rename(columns=lambda x: x.split(" ")[0], inplace=True)
# promo_pr.rename(columns=lambda x: x.split(" ")[0], inplace=True)

# display(reg_pr)  
# display(promo_pr)






######
# Сортировка цен


# сп

# hpp = pd.DataFrame()

# df = pd.DataFrame()
# min_values = reg_pr.min(axis=1)
# min_cols = reg_pr.idxmin(axis=1)
    
# df["сп"] = min_values.apply(lambda x: x if x < 40 else None)
# df["сп к"] = min_cols.apply(lambda x: x.split(" ")[0] if isinstance(x, str) else None)
    
# for col, row in min_cols.items():
#     reg_pr.at[col, row] = None
        
# hpp = pd.concat([hpp, df], axis=1)
# hpp.iloc[:, 1] = hpp.iloc[:, 1].where(~pd.isnull(hpp.iloc[:, 0]), None)
# display(hpp)


# рпц
regp_count = reg_pr.count(axis=1).max()
regp_sorted = pd.DataFrame()

for i in range(regp_count):
    df = pd.DataFrame()
    min_values = reg_pr.min(axis=1)
    min_cols = reg_pr.idxmin(axis=1)
    
    df["рпц " + str(i+1)] = min_values
    df["рпц к" + str(i+1)] = min_cols.apply(lambda x: x.split(" ")[0] if isinstance(x, str) else None)
    
    for col, row in min_cols.items():
        reg_pr.at[col, row] = None
        
    regp_sorted = pd.concat([regp_sorted, df], axis=1)
# display(regp_sorted)
    
    
# апц

promop_count = promo_pr.count(axis=1).max()
promop_sorted = pd.DataFrame()

for i in range(promop_count):
    df = pd.DataFrame()
    min_values = promo_pr.min(axis=1)
    min_cols = promo_pr.idxmin(axis=1)
    
    df["апц " + str(i+1)] = min_values
    df["апц к" + str(i+1)] = min_cols.apply(lambda x: x.split(" ")[0] if isinstance(x, str) else None)
    
    for col, row in min_cols.items():
        promo_pr.at[col, row] = None
        
    promop_sorted = pd.concat([promop_sorted, df], axis=1)
# display(promop_sorted)


f_report = pd.concat([f_report, regp_sorted, promop_sorted], axis=1)

# display(f_report)
# ########

# # rp_df.to_excel(final_name,
# #                     index=False)
# # print ('создан файл ' + final_name  + ' - ' + str(time.time() - start))




    
f_report.to_excel(final_name,
                    index=False)
print ('создан файл ' + final_name  + ' - ' + str(time.time() - start))

# # # # display(f_report)
# # # ##############################

создан файл final_s1.xlsx - 148.78562808036804


In [34]:
l_df_values = ['Цена по ДК', 'Цена по акции']
print(str(l_df_values[0])+"_l")

Цена по ДК_l
